> ![](https://raw.githubusercontent.com/SunnySuite/Sunny.jl/main/assets/sunny_logo.jpg)

_This is a tutorial for the [Sunny](https://github.com/SunnySuite/Sunny.jl/) package, which enables dynamical simulations of ordered and thermally disordered spins with dipole and higher order moments._

## **Spin Dynamics of the Heisenberg pyrochlore antiferromagnet and applications to MgCr2O4**
#### Author: Martin Mourigal (mourigal@gatech.edu)
#### Date: 09/03/2022
#### Version: Julia 1.8, Sunny updated 09/03/2022

In this tutorial, we will walk through a first example in Su(n)ny and calculate the spin dynamical properties of the Heisenberg pyrochlore antiferromagnet and apply this knowledge to MgCr2O4 and ZnCr2O4 which are known to approximate that model. Relevant publications are here:

[1] P. H. Conlon and J. T. Chalker, Phys. Rev. Lett. 102, 237206 (2009) (https://doi.org/10.1103/PhysRevLett.102.237206)

[2] P. H. Conlon and J. T. Chalker, Phys. Rev. B 81, 224413 (2010) (https://doi.org/10.1103/PhysRevB.81.224413)

[3] X. Bai, J. A. M. Paddison, _et al._ Phys. Rev. Lett. 122, 097201 (2019) (https://doi.org/10.1103/PhysRevLett.122.097201)

## **Setting up Julia with the necessary packages**

Important ressources to get started:   
https://github.com/SunnySuite/Sunny.jl/blob/main/GettingStarted.md          
https://sunnysuite.github.io/Sunny.jl/dev/library/#Library
https://github.com/ddahlbom/SunnyTutorials/blob/main/tutorials/FeI2_tutorial.ipynb

In [5]:
using Sunny # The main package
import ProgressMeter, Statistics, Formatting, StaticArrays, LinearAlgebra, DelimitedFiles, OffsetArrays, Random # Some useful peripheral packages
using GLMakie, Plots # Some useful plotting packages
Sunny.offline_viewers(); 

## **Setting up the Crystal Structure for the Model and MgCr2O4**

This is the way theorists think about the pyrochlore lattice, and we are going to input this by hand in our system. For convenience we pick a lattice parameter a = 6.28 Å (it's 2π!).

> ![](https://www.oist.jp/sites/default/files/photos/20200110-diagram-of-pyrochlore-lattice.jpg)

> _Picture Credits: Theory of Quantum Matter Unit, OIST_

#### "Theorist" Method ####

In [21]:
# === Define the crystal structure of a B-site pyrochlore lattice with only B-site atoms === 
lat_vecs = lattice_vectors(6.28, 6.28, 6.28, 90.0, 90.0, 90.0);  
bas_vecs = [  [0.87500000, 0.62500000, 0.37500000],
              [0.62500000, 0.12500000, 0.62500000],
              [0.87500000, 0.87500000, 0.12500000],  
              [0.62500000, 0.87500000, 0.37500000],  
              [0.87500000, 0.12500000, 0.87500000],  
              [0.62500000, 0.62500000, 0.12500000],  
              [0.87500000, 0.37500000, 0.62500000],  
              [0.62500000, 0.37500000, 0.87500000],  
              [0.37500000, 0.62500000, 0.87500000],  
              [0.12500000, 0.12500000, 0.12500000],  
              [0.37500000, 0.87500000, 0.62500000],  
              [0.12500000, 0.87500000, 0.87500000],  
              [0.37500000, 0.12500000, 0.37500000],  
              [0.12500000, 0.62500000, 0.62500000],  
              [0.37500000, 0.37500000, 0.12500000],  
              [0.12500000, 0.37500000, 0.37500000]];
bas_typs = ["B","B","B","B", "B","B","B","B", "B","B","B","B", "B","B","B","B"]; 
xtal_pyro   = Crystal(lat_vecs, bas_vecs; types=bas_typs); # We will call this crystal the Theoretical Pyrochlore

# === Return crystalographic information gathered by Sunny about the crystal structure === 
display(xtal_pyro ); # Sunny can identify the space-group and point-group for this system. Here Fd-3m
view_crystal(xtal_pyro, 3.2) # Sunny can draw the crystal structure.

Crystal
HM symbol 'F d -3 m' (227)
Lattice params a=6.28, b=6.28, c=6.28, α=90°, β=90°, γ=90°
Cell volume 247.7
Type 'B', Wyckoff 16c (point group '.-3m'):
   1. [0.875, 0.625, 0.375]
   2. [0.625, 0.125, 0.625]
   3. [0.875, 0.875, 0.125]
   4. [0.625, 0.875, 0.375]
   5. [0.875, 0.125, 0.875]
   6. [0.625, 0.625, 0.125]
   7. [0.875, 0.375, 0.625]
   8. [0.625, 0.375, 0.875]
   9. [0.375, 0.625, 0.875]
   10. [0.125, 0.125, 0.125]
   11. [0.375, 0.875, 0.625]
   12. [0.125, 0.875, 0.875]
   13. [0.375, 0.125, 0.375]
   14. [0.125, 0.625, 0.625]
   15. [0.375, 0.375, 0.125]
   16. [0.125, 0.375, 0.375]


SunnyViewer(...)

Off course, a real crystal is more complicated and we will know construct the system using the actual .cif file of MgCr2O4 from ICSD. The first way you could do that is importing by hand the data from a .cif file, but this can be dangerous, as shown below:

#### "Experimentalist" Method #1 (Incorrect) ####

In [34]:
# === Define the crystal structure of MgCr2O4 by copying the info from a .cif file === 
lat_vecs     = lattice_vectors(8.3342, 8.3342, 8.3342, 90.0, 90.0, 90.0);  
bas_vecs     = [  [0.12500, 0.12500, 0.12500],
              [0.50000, 0.50000, 0.50000],
              [0.26070, 0.26070, 0.26070]]; 
bas_typs     = ["Mg","Cr","O"];
xtal_mgcro_1 = Crystal(lat_vecs, bas_vecs; types=bas_typs); 
display(xtal_mgcro_1); 

Crystal
HM symbol 'R 3 m' (160)
Lattice params a=8.334, b=8.334, c=8.334, α=90°, β=90°, γ=90°
Cell volume 578.9
Type 'Mg', Wyckoff 3a (point group '3m'):
   1. [0.125, 0.125, 0.125]
Type 'Cr', Wyckoff 3a (point group '3m'):
   2. [0.5, 0.5, 0.5]
Type 'O', Wyckoff 3a (point group '3m'):
   3. [0.2607, 0.2607, 0.2607]


Here, obviously, we are not getting the right space group for MgCr2O4. This can be achieved by modifying the input to include the space group and the setting.

#### "Experimentalist" Method #2 (Correct) ####

In [54]:
# === Define the crystal structure of MgCr2O4 by copying the info from a .cif file INCLUDING space group and setting === 

lat_vecs     = lattice_vectors(8.3342, 8.3342, 8.3342, 90.0, 90.0, 90.0);  
bas_vecs     = [ [0.12500, 0.12500, 0.12500],
                [0.50000, 0.50000, 0.50000],
                [0.26070, 0.26070, 0.26070]]; 
bas_typs     = ["Mg","Cr","O"];
lat_spg      = 227; # Space Group Number 
lat_set      = "2"; # Space Group setting
xtal_mgcro_2 = Crystal(lat_vecs, bas_vecs, lat_spg; types=bas_typs, setting=lat_set); 
display(xtal_mgcro_2);
view_crystal(xtal_mgcro_2, 3.2) # Sunny can draw the crystal structure. 

Crystal
HM symbol 'F d -3 m' (227)
Lattice params a=8.334, b=8.334, c=8.334, α=90°, β=90°, γ=90°
Cell volume 578.9
Type 'Mg', Wyckoff 8b (point group '-43m'):
   1. [0.125, 0.125, 0.125]
   2. [0.625, 0.625, 0.125]
   3. [0.875, 0.375, 0.375]
   4. [0.375, 0.875, 0.375]
   5. [0.625, 0.125, 0.625]
   6. [0.125, 0.625, 0.625]
   7. [0.375, 0.375, 0.875]
   8. [0.875, 0.875, 0.875]
Type 'Cr', Wyckoff 16c (point group '.-3m'):
   9. [0.5, 0, 0]
   10. [0.75, 0.25, 0]
   11. [0, 0.5, 0]
   12. [0.25, 0.75, 0]
   13. [0.75, 0, 0.25]
   14. [0.5, 0.25, 0.25]
   15. [0.25, 0.5, 0.25]
   16. [0, 0.75, 0.25]
   17. [0, 0, 0.5]
   18. [0.25, 0.25, 0.5]
   19. [0.5, 0.5, 0.5]
   20. [0.75, 0.75, 0.5]
   21. [0.25, 0, 0.75]
   22. [0, 0.25, 0.75]
   23. [0.75, 0.5, 0.75]
   24. [0.5, 0.75, 0.75]
Type 'O', Wyckoff 32e (point group '.3m'):
   25. [0.7393, 0.0107, 0.0107]
   26. [0.5107, 0.2393, 0.0107]
   27. [0.2393, 0.5107, 0.0107]
   28. [0.0107, 0.7393, 0.0107]
   29. [0.5107, 0.0107, 0.2393]
  

SunnyViewer(...)

This is correct, but at this point we might as well import the cif file directly, but sometimes the files are not formatted correctly!

#### "Experimentalist" Method #3 (Correct but watchout for bad .cif files) ####

In [43]:
# === Define the crystal structure of MgCr2O4 by importing a cif file === 
xtal_mgcro_3 = Crystal("MgCr2O4_9006236_1994.cif",symprec=0.001);
display(xtal_mgcro_3); 

ArgumentError: ArgumentError: column name :_atom_site_type_symbol not found in the data frame

In [ ]:
Oh no! Our .cif file is not formatted correctly... Well, we can always work with our method where we enter the .cif information by hand. But just to show you:

In [51]:
# === Define the crystal structure of MgCr2O4 by importing a cif file === 
xtal_mgcro_3 = Crystal("MgCr2O4_160953_2009.cif",symprec=0.001);
display(xtal_mgcro_3);

Crystal
HM symbol 'F d -3 m' (227)
Lattice params a=8.333, b=8.333, c=8.333, α=90°, β=90°, γ=90°
Cell volume 578.6
Type 'Mg2+', Wyckoff 8b (point group '-43m'):
   1. [0.125, 0.125, 0.125]
   2. [0.625, 0.625, 0.125]
   3. [0.875, 0.375, 0.375]
   4. [0.375, 0.875, 0.375]
   5. [0.625, 0.125, 0.625]
   6. [0.125, 0.625, 0.625]
   7. [0.375, 0.375, 0.875]
   8. [0.875, 0.875, 0.875]
Type 'Cr3+', Wyckoff 16c (point group '.-3m'):
   9. [0.5, 0, 0]
   10. [0.75, 0.25, 0]
   11. [0, 0.5, 0]
   12. [0.25, 0.75, 0]
   13. [0.75, 0, 0.25]
   14. [0.5, 0.25, 0.25]
   15. [0.25, 0.5, 0.25]
   16. [0, 0.75, 0.25]
   17. [0, 0, 0.5]
   18. [0.25, 0.25, 0.5]
   19. [0.5, 0.5, 0.5]
   20. [0.75, 0.75, 0.5]
   21. [0.25, 0, 0.75]
   22. [0, 0.25, 0.75]
   23. [0.75, 0.5, 0.75]
   24. [0.5, 0.75, 0.75]
Type 'O2-', Wyckoff 32e (point group '.3m'):
   25. [0.7388, 0.0112, 0.0112]
   26. [0.5112, 0.2388, 0.0112]
   27. [0.2388, 0.5112, 0.0112]
   28. [0.0112, 0.7388, 0.0112]
   29. [0.5112, 0.0112, 0.23

SunnyViewer(...)

#### Subcrystals ####

Finally, we constructing a spin Hamiltonian, it is useful to only consider the magnetic atoms in the unit-cell while keeping the symmetry correct for the determination of exchange and g-factor anisotropies. This is achieved below:

In [88]:
# === Define a sub crystal === 
xtal_mgcro = subcrystal(xtal_mgcro_2,"Cr");
display(xtal_mgcro); # Note the warming about renumbering atoms

Crystal
HM symbol 'F d -3 m' (227)
Lattice params a=8.334, b=8.334, c=8.334, α=90°, β=90°, γ=90°
Cell volume 578.9
Type 'Cr', Wyckoff 16c (point group '.-3m'):
   1. [0.5, 0, 0]
   2. [0.75, 0.25, 0]
   3. [0, 0.5, 0]
   4. [0.25, 0.75, 0]
   5. [0.75, 0, 0.25]
   6. [0.5, 0.25, 0.25]
   7. [0.25, 0.5, 0.25]
   8. [0, 0.75, 0.25]
   9. [0, 0, 0.5]
   10. [0.25, 0.25, 0.5]
   11. [0.5, 0.5, 0.5]
   12. [0.75, 0.75, 0.5]
   13. [0.25, 0, 0.75]
   14. [0, 0.25, 0.75]
   15. [0.75, 0.5, 0.75]
   16. [0.5, 0.75, 0.75]


WARNING, atoms are being renumbered.


## **Setting up the Single-Ion and Exchange Properties of our System** ##

Sunny can determine exchange and g-tensor anisotropies. This is achieved as follows:

#### Symmetry Analysis of Exchange and Single-Ion anisotropies ####

In [89]:
display("======== MgCr2O4 Exchange Interactions to Third Neighbor ========")
print_bond_table(xtal_mgcro, 6.0); # Print g-factor of Cr site and exchange interactions to distance of 6.0Å

"======== MgCr2O4 Exchange Interactions to Third Neighbor ========"

Atom 1, type 'Cr', position [0.5, 0, 0], multiplicity 16
Allowed single-ion anisotropy or g-tensor: | A  B  B |
                                           | B  A  B |
                                           | B  B  A |

Bond(1, 2, [0, 0, 0])
Distance 2.9466, coordination 6
Connects 'Cr' at [0.5, 0, 0] to 'Cr' at [0.75, 0.25, 0]
Allowed exchange matrix: | A  C -D |
                         | C  A -D |
                         | D  D  B |
Allowed DM vector: [-D D 0]

Bond(1, 7, [0, 0, 0])
Distance 5.1036, coordination 12
Connects 'Cr' at [0.5, 0, 0] to 'Cr' at [0.25, 0.5, 0.25]
Allowed exchange matrix: |   A  C-E  D-F |
                         | C+E    B -C+E |
                         | D+F -C-E    A |
Allowed DM vector: [E F -E]

Bond(1, 3, [0, 0, 0])
Distance 5.8932, coordination 6
Connects 'Cr' at [0.5, 0, 0] to 'Cr' at [0, 0.5, 0]
Allowed exchange matrix: | A  D  C |
                         | D  A  C |
                         | C  C  B |

Bond(1, 3, [1, 0, 0])
Distance 5.8932,

In [72]:
display("======== MgCr2O4 Allowed single-ion anisotropy ========")
print_allowed_anisotropy(xtal_mgcro, 1);

"======== MgCr2O4 Allowed single-ion anisotropy ========"

# Stevens operators at various orders k
𝒪₂ = stevens_operators(N, 2)
𝒪₄ = stevens_operators(N, 4)
𝒪₆ = stevens_operators(N, 6)

# Allowed anisotropies
A₂(𝒪₂[-2]+2𝒪₂[-1]+2𝒪₂[1])
A₄(7𝒪₄[-3]+2𝒪₄[-2]-𝒪₄[-1]-𝒪₄[1]-7𝒪₄[3]) + B₄(𝒪₄[0]+5𝒪₄[4])
A₆(11𝒪₆[-6]+8𝒪₆[-3]-𝒪₆[-2]+8𝒪₆[-1]+8𝒪₆[1]-8𝒪₆[3]) + B₆((33/2)𝒪₆[-5]+(-9/2)𝒪₆[-3]+4𝒪₆[-2]+𝒪₆[-1]+𝒪₆[1]+(9/2)𝒪₆[3]+(33/2)𝒪₆[5]) + C₆(𝒪₆[0]-21𝒪₆[4])


In [68]:
display("======== Pyrochlore Lattice Exchange Interactions to Third Neighbor ========")
print_bond_table(xtal_pyro, 4.5); # Print g-factor of pyrochlore and exchange interactions to distance of 4.0Å

"======== Pyrochlore Lattice Exchange Interactions to Third Neighbor ========"

Atom 1, type 'B', position [0.875, 0.625, 0.375], multiplicity 16
Allowed single-ion anisotropy or g-tensor: | A  B -B |
                                           | B  A  B |
                                           |-B  B  A |

Bond(1, 3, [0, 0, 0])
Distance 2.2203, coordination 6
Connects 'B' at [0.875, 0.625, 0.375] to 'B' at [0.875, 0.875, 0.125]
Allowed exchange matrix: | A -D  D |
                         | D  B  C |
                         |-D  C  B |
Allowed DM vector: [0 -D -D]

Bond(1, 2, [0, 0, 0])
Distance 3.8457, coordination 12
Connects 'B' at [0.875, 0.625, 0.375] to 'B' at [0.625, 0.125, 0.625]
Allowed exchange matrix: |   A  C-E  D-F |
                         | C+E    B -C+E |
                         | D+F -C-E    A |
Allowed DM vector: [E F -E]

Bond(2, 6, [0, 0, 0])
Distance 4.4406, coordination 6
Connects 'B' at [0.625, 0.125, 0.625] to 'B' at [0.625, 0.625, 0.125]
Allowed exchange matrix: | A  D  D |
                         | D  B  C |
                      

Note that the exchange anisotropies of the Pyrochlore model are slightly different from that of MgCr2O4, due to the role of Mg and O in the bonds. Also note that the program indentifies correctly the two inequivalent bonds 3a and 3b having the same length. A question may arises to know which bond is J3a and which is J3b, let's plot the structure:

In [98]:
view_crystal(xtal_mgcro, 5.9) # Sunny can draw the crystal structure. 

SunnyViewer(...)

The above plot shows that the second interaction (cyan color) with the distance of 5.89Å is in fact the one hopping through a Chromium site, meaning it is J3a! So we need to be careful with that later.

#### Building the local Hilbert Space for our system ####

In [75]:
# === Define Local Hilbert Space for Cr ===
lhs_Cr  = SiteInfo(1; N=0, g=2.0, spin_rescaling=3/2);  # Local Hilbert Space for a spin-3/2 treated classically (N=0) and with g=2. If all the operators of the local S=3/2 operators have to be retained, this corresponds to N=4.

#### Building the exchange interactions for our system ####

In [101]:
# === Define Values of Exchange Interactions ===
val_J1      = 3.27; # value of J1 in meV from Bai's PRL paper
val_J_pyro  = [1.00,0.000,0.000,0.000]*val_J1; # pure nearest neighbor pyrochlore
val_J_mgcro = [1.00,0.080,0.100,0.050]*val_J1; # further neighbor pyrochlore relevant for MgCr2O4

# === Build list of exchange interactions for our system ===
exint_pyro  = [heisenberg(val_J_pyro[1],  Bond(1, 3, [0, 0, 0]),"J1")];
exint_mgcro = [heisenberg(val_J_mgcro[1], Bond(1, 2, [0, 0, 0]),"J1"),
               heisenberg(val_J_mgcro[2], Bond(1, 7, [0, 0, 0]),"J2"),
               heisenberg(val_J_mgcro[3], Bond(1, 3, [0, 0, 0]),"J3a"),   # Watch out for which is J3a see above plots
               heisenberg(val_J_mgcro[4], Bond(1, 3, [0, 1, 0]),"J3b")];   # Watch out for which is J3b see above plots                                                                   



